In [1]:
import os
import sys
import torch.backends.cudnn as cudnn

module_path = os.path.abspath(os.path.join('../../python'))
if module_path not in sys.path:
    sys.path.append(module_path)


from cvpr2018.utils.utils import register_logger
from encoder.clip_encoder import ClipEncoder

/home/ubuntu/anaconda3/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
log_every = 50  # log the writing of clips every n steps
log_file = None  # set logging file
num_workers = 4  # define the number of workers used for loading the videos

cudnn.benchmark = True
register_logger(log_file=log_file)

clip_length = 16  # define the length of each input sample
frame_interval = 1 # define the sampling interval between framesq
batch_size = 4


In [3]:
anomalous_videos = '/home/ubuntu/repos/llm-rag/data/Anomaly-Videos-Part-1/test'
encoder = ClipEncoder(anomalous_videos, clip_length, frame_interval, batch_size, num_workers)



2024-04-16 09:20:32,402 Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-04-16 09:20:32,440 Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-04-16 09:20:32,450 Found 1 video files in /home/ubuntu/repos/llm-rag/data/Anomaly-Videos-Part-1/test


100%|██████████| 1/1 [00:01<00:00,  1.00s/it]


In [4]:
frames = encoder.export_tensor_to_np()

/home/ubuntu/anaconda3/envs/llava/lib/python3.10/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


In [5]:
# from transformers import AutoProcessor, LlavaForConditionalGeneration
# from transformers import BitsAndBytesConfig
# import torch

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16
# )

# model_id = "llava-hf/llava-1.5-7b-hf"

# processor = AutoProcessor.from_pretrained(model_id)
# model = LlavaForConditionalGeneration.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")



In [6]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration

from transformers import BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    #load_in_8bit=True
)

model_id = "llava-hf/llava-v1.6-mistral-7b-hf"

processor = LlavaNextProcessor.from_pretrained(model_id)
model = LlavaNextForConditionalGeneration.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2024-04-16 09:20:53,623 We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
2024-04-16 09:20:54,110 The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.37s/it]


In [7]:
prompt = "[INST] <image>\nDescribe the scene very briefly. Pay special attention to whether an assault takes place. [/INST]:"

prompts = [prompt for _ in range(len(frames))]

print(len(prompts), len(frames))

inputs = processor(prompts, images=frames, padding=True, return_tensors="pt").to("cuda")
for k,v in inputs.items():
  print(k,v.shape)

170 170
input_ids torch.Size([170, 28])
attention_mask torch.Size([170, 28])
pixel_values torch.Size([170, 3, 3, 336, 336])
image_sizes torch.Size([170, 2])


In [8]:
print(len(inputs["input_ids"]))

170


In [9]:
batch_size = 10  # Set your desired batch size
generated_text = []

for i in range(0, len(inputs["input_ids"]), batch_size):
    print(i)
    batch_inputs = {key: value[i:i+batch_size] for key, value in inputs.items()}
    output = model.generate(**batch_inputs, max_new_tokens=100)
    generated_text.extend(processor.batch_decode(output, skip_special_tokens=True))

for text in generated_text:
    print(text.split("ASSISTANT:")[-1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


10


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

140


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


150


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


160
[INST]  
Describe the scene very briefly. Pay special attention to whether an assault takes place. [/INST]: The image shows an interior space that appears to be a room with a high ceiling and architectural features. There are two individuals in the room; one is standing and the other is seated at a table. The seated person is facing the standing person, and it seems like they are engaged in a conversation or an activity. There is no clear indication of an assault taking place in the image. The room has a formal or ceremonial atmosphere, with a large portrait or painting on the wall behind the seated
[INST]  
Describe the scene very briefly. Pay special attention to whether an assault takes place. [/INST]: The image shows an interior space that appears to be a room with a high ceiling and architectural features. There are two individuals in the room; one is standing and the other is seated at a table. The seated person is facing away from the camera, and the standing person is facin

In [10]:
# for frame in frames[:2]:
#   display(Image.fromarray(frame))